<a href="https://colab.research.google.com/github/stygian2a/RoombaGym/blob/master/Roomba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO


In [2]:
!pip uninstall keras -y
!pip install keras==2.2.4 

!pip uninstall tensorflow -y
!pip install tensorflow==1.13.1 

#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Uninstalling Keras-2.2.4:
  Successfully uninstalled Keras-2.2.4
  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
  Using cached https://files.pythonhosted.org/packages/77/63/a9fa76de8dffe7455304c4ed635be4aa9c0bacef6e0633d87d5f54530c5c/tensorflow-1.13.1-cp36-cp36m-manylinux1_x86_64.whl


In [3]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!pip install keras-rl

!wget https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.2.0)
--2020-02-21 17:39:15--  https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11819 (12K) [text/plain]
Saving to: ‘rendering.py.2’

rendering.py.2      100%[===================>]  11.54K  --.-KB/s    in 0s      

2020-02-21 17:39:15 (98.2 MB/s) - ‘rendering.py.2’ saved [11819/11819]



In [4]:
import gym
from gym import spaces
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from time import sleep
from scipy.stats import multivariate_normal

from IPython import display as ipythondisplay

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [5]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:

class Roomba():
  def __init__(self, pos, direction, radius, battery, turn_angle=np.pi/9):
    self.pos = np.array(pos)
    self.direction = direction
    self.radius = radius
    self.turn_angle = turn_angle
    self.battery = battery
    self.max_battery = battery

  def set_pos(self, pos):
    self.pos = np.array(pos)

  def get_forward(self):
    new_x = np.sin(self.direction) + self.pos[0]
    new_y = np.cos(self.direction) + self.pos[1]
    return [new_x, new_y]

  def get_backward(self):
    new_x = np.sin(self.direction+np.pi) + self.pos[0]
    new_y = np.cos(self.direction+np.pi) + self.pos[1]
    return [new_x, new_y]

  def turn_right(self):
    self.direction = (self.direction - self.turn_angle) % (2*np.pi)

  def turn_left(self):
    self.direction = (self.direction + self.turn_angle) % (2*np.pi)

  def decrease_battery(self):
    self.battery -=1

  def recharge_battery(self):
    self.battery = self.max_battery

  def get_life(self):
    return self.battery / self.max_battery


  #https://stackoverflow.com/questions/49551440/python-all-points-on-circle-given-radius-and-center
  def get_cases_underneath(self, xmax, ymax, pos = None):
    if pos == None:
      pos = self.pos
    radius = self.radius
    x0 = pos[0] - 0.5
    y0 = pos[1] - 0.5

    L = []
    x_ = np.arange(x0 - radius - 1, x0 + radius + 1, dtype=int)
    y_ = np.arange(y0 - radius - 1, y0 + radius + 1, dtype=int)
    x, y = np.where((x_[:,np.newaxis] - x0)**2 + (y_ - y0)**2 <= radius**2)
    for x, y in zip(x_[x], y_[y]):
      if x>=0 and y>=0 and x<xmax and y<ymax:
        L.append((x, y))
    return L

In [0]:
class RoombaEnv(gym.Env):

  metadata = {
      'render.modes': ['human', 'rgb_array'],
      'video.frames_per_second' : 50
  }

  def __init__(self, height, width, battery, roomba_radius=2):
    super(RoombaEnv, self).__init__()

    self.action_space = spaces.Discrete(4) #Forward, Backwards, Right, Left

    """                                      
    self.observation_space = spaces.Tuple([
        spaces.Box(low=0, high=4, shape=(height, width), dtype=np.float32), #1=cleanable, 2=wall, 3=charging station
        spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([height, width, 2*np.pi, 1]), dtype=np.float32)
    ])
    """
    #self.observation_space = spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([height, width, 2*np.pi, 1]), dtype=np.float32)
    self.observation_space = spaces.Box(low=np.array(np.zeros(4+height*width)), high=np.concatenate([np.array([height, width, 2*np.pi, 1]), np.ones(height*width)]), dtype=np.float32)

    self.width = width
    self.height = height
    self.battery = battery
    self.roomba_radius = roomba_radius

    self.cases_indexes = np.array( [(i,j) for i in range(height) for j in range(width)] )
    self.dirt_probabilities = multivariate_normal.pdf(self.cases_indexes, mean=[height/2,width/2], cov=10*np.diag([1,1]))
    self.dirt_probabilities = self.dirt_probabilities / np.sum(self.dirt_probabilities)

    self.room = np.ones((height, width))
    #obstacles = [[5,5], [5,6], [5,7], [5,8], [6,5], [7,5]]
    obstacles = []
    for obstIndex in obstacles:
      self.room[obstIndex[0],obstIndex[1]]= 2
    for i in range(4):
      for j in range(4):
        self.room[height-i-1, width-j-1] = 3
    self.viewer = None
    self.init_env()

  def init_env(self):
    self.roomba = Roomba( [self.height-3, self.width-3], -np.pi/2, self.roomba_radius, self.battery)
    self.done = False
    self.iteration = 0
    self.sum_rewards = 0
    self.dirty = np.zeros((self.height, self.width))


  def clean_floor(self):
    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width)
    reward = 0
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 1:
        reward += min(1,self.dirty[case_underneath])
        self.dirty[case_underneath] = 0

      if self.room[case_underneath] == 3:
        self.roomba.recharge_battery()
    return reward

  def soil_floor(self):
    i = np.random.choice(self.height * self.width, p=self.dirt_probabilities)
    self.dirty[self.cases_indexes[i][0], self.cases_indexes[i][1]] = 1

  def step(self, action):
    self.iteration+=1
    #if random.random() <0.1:
    #  action = random.randint(0,3)
    #current_reward = 1
    current_reward = 0
    if action == 0:
      new_pos = self.roomba.get_forward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)
      #else:
      #  current_reward += -10

    elif action == 1:
      new_pos = self.roomba.get_backward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)
      #else:
      #  current_reward += -10

    elif action == 2:
      self.roomba.turn_right()

    elif action == 3:
      self.roomba.turn_left()
    self.roomba.decrease_battery()
      
    current_reward += self.clean_floor()

    self.soil_floor()

    if self.roomba.battery <= 0:
      current_reward += -100

    self.sum_rewards += current_reward

    if self.iteration>=10000 or self.roomba.battery <= 0:
      self.done = True
      
    return self.get_observation(),current_reward,self.done,{}

  def is_safe(self, new_pos):
    x = new_pos[0]
    y = new_pos[1]

    if y < 0 or x < 0 or x >= self.height or y >= self.width:
      return False

    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width, new_pos)
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 2:
        return False

    return True

  def get_observation(self):
    #return np.array([self.roomba.pos[0]/self.height, self.roomba.pos[1]/self.width, self.roomba.direction, self.roomba.get_life()])
    return np.concatenate([np.array([self.roomba.pos[0]/self.height, self.roomba.pos[1]/self.width, self.roomba.direction, self.roomba.get_life()]), self.dirty.flatten()])

    #return (self.dirty, np.array([self.roomba.pos[0], self.roomba.pos[1], self.roomba.direction, self.roomba.get_life()]))

  def reset(self):
    self.init_env()
    return self.get_observation()

  def render(self, mode='human', close=False):
    scale = 6

    screen_width = scale*self.width
    screen_height = scale*self.height


    if self.viewer is None:
      #TODO: remove when it works
      import importlib
      import rendering
      importlib.reload(rendering)

      self.viewer = rendering.Viewer(screen_width, screen_height)

      self.cases = []
      for i in range(self.height):
        self.cases.append([])
        for j in range(self.width):
          x0 = i*scale
          y0 = j*scale
          x1 = (i+1)*scale
          y1 = (j+1)*scale
          case = rendering.FilledPolygon([(y0,x0), (y0,x1), (y1,x1), (y1,x0)])
          self.cases[i].append(case)
          self.viewer.add_geom(case)
      
      circle = rendering.make_circle(radius=self.roomba.radius*scale)
      circle.set_color(1,0,0)
      self.circle=circle
      direction_circle = rendering.make_circle(radius=self.roomba.radius*scale/2) #color?
      direction_circle.add_attr(rendering.Transform(translation=(self.roomba.radius*scale/2,0)))

      tracker = rendering.Compound([circle, direction_circle])

      self.tracker_trans = rendering.Transform()
      tracker.add_attr(self.tracker_trans)
      self.viewer.add_geom(tracker)

      
    for i in range(self.height):
        for j in range(self.width):
          if self.room[i,j] == 0:
            self.cases[i][j].set_color(1,1,1)
          elif self.room[i,j] == 1:
            self.cases[i][j].set_color(1 - 0.5 * min(1,self.dirty[i,j]),1 - 0.5 * min(1,self.dirty[i,j]),1 - 0.5 * min(1,self.dirty[i,j]))
          elif self.room[i,j] == 2:
            self.cases[i][j].set_color(0,0,0)
          elif self.room[i,j] == 3:
            self.cases[i][j].set_color(0.5,0.5,0)


    self.tracker_trans.set_translation(self.roomba.pos[1]*scale, self.roomba.pos[0]*scale)
    self.tracker_trans.set_rotation(self.roomba.direction)
    self.circle.set_color(self.roomba.get_life(),0,0)


    return self.viewer.render(return_rgb_array = mode=='rgb_array')

In [0]:
#TODO : ADD SOME EFFECT FOR TURNING INSTEAD OF TAKING OLD POS

Height = 20
Width = 20
battery = 500
pad = 3

def closestDirt(x, y, mat):
  d = float('inf')
  for k in range(Height): 
    for l in range(Width):  
        if (mat[k][l] == 1): 
            d = min(d, abs(x - k) + abs(y - l))
  return d

def feature1(state, action):
  dirtMap = state[4:].reshape(20,20)
  pos = action()
  if pos==None:
    pos=state[:2].copy()
    pos[0]*=Height
    pos[1]*=Width
  x_min = max(0,int(pos[0])-pad)
  x_max = min(Height,int(pos[0])+pad)
  y_min = max(0,int(pos[1])-pad)
  y_max = min(Width,int(pos[1])+pad)
  AAA = np.sum(dirtMap[x_min:x_max, y_min:y_max])/(pad*pad)
  return AAA

def feature2(state, action):
  charger = np.array([Height-3,Width-3])
  life = state[3]
  pos = action()
  if pos==None:
    pos=state[:2].copy()
    pos[0]*=1
    pos[1]*=1
  else:
    life -= - 1/500
  distManhattan = charger-pos
  distManhattan = np.sum(np.abs(distManhattan))
  #print(min((1-life)/distManhattan,1))
  return min((1-life)/distManhattan,1)

def feature3(state, action):
  dirtMap = state[4:].reshape(20,20)
  pos = action()
  if pos==None:
    pos=state[:2].copy()
    pos[0]*=Height
    pos[1]*=Width
  BBB = closestDirt(int(pos[0]),int(pos[1]), dirtMap)/(Height+Width)
  print(BBB)
  return BBB

def feature4(state, action):
  return 1

In [73]:
env = wrap_env(RoombaEnv(20, 20, 500, 2))

# Hyperparameters
alpha = 0.1
gamma = 0.9
epsilon = 0.5

weights = [0, 0, 0, 0]
F = [feature1, feature2]

def getQValue(state, action):
  qv = 0
  for i, feature in enumerate(F):
      qv = qv + weights[i] * feature(state, action)
  return qv


possible_actions = [env.roomba.get_forward, env.roomba.get_backward, env.roomba.turn_right, env.roomba.turn_left]
for e in range(10):
  done = False
  state = env.reset()
  while not done:
    if random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()
    else:
        q = np.array([getQValue(state, a) for a in possible_actions])
        action = np.argmax(q)

    next_state, reward, done, info = env.step(action)

    action = possible_actions[action]

    next_values = [getQValue(next_state, a) for a in possible_actions]
    next_max = max(next_values)

    alphadiff = alpha * ((reward + gamma * next_max) - getQValue(state, action))
    
    for i, feature in enumerate(F):
        weights[i] = weights[i] + alphadiff * feature(state, action)

    state = next_state

state = env.reset()
while not done:
  env.render()
  q = np.array([getQValue(state, a) for a in possible_actions])
  action = np.argmax(q)

  next_state, reward, done, info = env.step(action)

  action = possible_actions[action]

  next_values = [getQValue(next_state, a) for a in possible_actions]
  next_max = max(next_values)

  alphadiff = alpha * ((reward + gamma * next_max) - getQValue(state, action))
  
  for i, feature in enumerate(F):
      weights[i] = weights[i] + alphadiff * feature(state, action)

  state = next_state

env.close()
show_video()

In [75]:
weights

[-6.610074421558538, -8.71723161394409, 0, 0]

In [12]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action)
    
    if done:
      break;
            
env.close()
show_video()

If the goal is to train on an unchanging room, then no need for giving walls/charging station as input? He will learn automatically?


In [0]:

import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory



env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse

nb_actions = env.action_space.n


# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=60000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

# After training is done, we save the final weights.
dqn.save_weights('dqn_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=False)


In [0]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [0]:
! rm -rf video/

#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
#observation = env.reset()

dqn.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

In [0]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory



# Get the environment and extract the number of actions.
env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse


nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
#model = Sequential()
#model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
#model.add(Dense(nb_actions))
#model.add(Activation('softmax'))

# Option 2: deep network
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=100000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=5000, nb_steps_warmup=10000, train_interval=50, elite_frac=0.05)
cem.compile()

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=200000, visualize=False, verbose=1)

# After training is done, we save the best weights.
cem.save_weights('cem_roomba_params.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=False)

In [0]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

cem.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

In [0]:
random.random()

In [0]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy



# Get the environment and extract the number of actions.
env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10000, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=False)

In [0]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

sarsa.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

In [0]:
np.random.choice(400, p=dirt_probabilities)

In [0]:
dirt_probabilities.shape